In [1]:
!nvidia-smi

Thu Jul  1 16:09:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip3 install transformers

     |████████████████████████████████| 2.5MB 28.0MB/s 
     |████████████████████████████████| 3.3MB 51.4MB/s 
     |████████████████████████████████| 901kB 48.2MB/s 


In [3]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch 
from transformers import BertModel,BertConfig,AutoModel,AutoTokenizer,AdamW,get_linear_schedule_with_warmup,get_constant_schedule
import pandas as pd 
import numpy as np 
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
from torch import nn,optim 
import seaborn as sns 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score,f1_score
# from fairseq.models.roberta import RobertaModel
# from fairseq.data.encoders.fastbpe import fastBPE
import random
import os 
from matplotlib import pyplot as plt 
from torch.nn.utils.rnn import pad_sequence
import pickle
from sklearn.metrics import accuracy_score

In [5]:
!cp /content/drive/MyDrive/Viettel3/train_n.csv /content
!cp /content/drive/MyDrive/Viettel3/test_n.csv /content

In [6]:
# train=pd.read_csv('train_n.csv')
test=pd.read_csv('test_n.csv')

In [7]:
phobert=AutoModel.from_pretrained("vinai/phobert-base")
tokenizer=AutoTokenizer.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
val=pd.DataFrame()
random.seed(42)
n_classes=len(set(train.label))
for i in range(n_classes):
    temp=train[train.label==i]
    temp=temp.sample(int(len(temp)*0.2))
    val=pd.concat([val,temp],axis=0)

train=train[~train.index.isin(val.index)]
train.to_csv("train.csv",index=False)
val.to_csv("val.csv",index=False)

In [ ]:
print(len(train))
print(len(val))

27012
6747


In [ ]:
# !cp /content/train.csv /content/drive/MyDrive/Viettel3/Bert
# !cp /content/val.csv /content/drive/MyDrive/Viettel3/Bert

In [10]:
!cp /content/drive/MyDrive/Viettel3/Bert/val.csv /content
!cp /content/drive/MyDrive/Viettel3/Bert/train.csv /content

In [11]:
# train=pd.read_csv('train.csv')
val=pd.read_csv('val.csv')

In [12]:
labels={
0:"Van hoa",
1:"Chinh tri Xa hoi",
2:"Khoa hoc",
3:"Phap luat",
4:"Vi tinh",
5:"Suc khoe",
6:"Kinh doanh",
7:"The thao",
8:"The gioi",
9:"Doi song"
}

In [13]:
max_sequence_length=256

def encode_data(data):
  result=[]
  for text in data.text.values:
    temp=tokenizer.encode(text)
    if len(temp)<max_sequence_length:
      temp+=[1]*(max_sequence_length-len(temp))
    else:
      t1=temp[:64]
      t2=temp[-192:]
      temp=t1+t2
    result.append(temp)
  return result

# train['data']=encode_data(train)
test['data']=encode_data(test)
val['data']=encode_data(val)

Token indices sequence length is longer than the specified maximum sequence length for this model (417 > 256). Running this sequence through the model will result in indexing errors


In [14]:
# train_dataset_tensor=torch.utils.data.TensorDataset(torch.tensor(train['data'],dtype=torch.long),torch.tensor(train['label'],dtype=torch.long))
test_dataset_tensor=torch.utils.data.TensorDataset(torch.tensor(test['data'],dtype=torch.long),torch.tensor(test['label'],dtype=torch.long))
val_dataset_tensor=torch.utils.data.TensorDataset(torch.tensor(val['data'],dtype=torch.long),torch.tensor(val['label'],dtype=torch.long))

In [15]:
batch_size=16
# train_loader=DataLoader(train_dataset_tensor,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_dataset_tensor,batch_size=batch_size,shuffle=True)
val_loader=DataLoader(val_dataset_tensor,batch_size=batch_size,shuffle=True)

In [18]:
# print(len(train_loader))
print(len(test_loader))
print(len(val_loader))

3149
422


In [19]:
class Model(nn.Module):
  def __init__(self,n_classes,drop_prob=0.2):
    super(Model,self).__init__()
    self.n_classes=n_classes
    self.bert=phobert
    self.lstm=nn.LSTM(input_size=self.bert.config.hidden_size,
                      hidden_size=512,
                      batch_first=True,
                      num_layers=2,
                      bidirectional=True)
    self.linear=nn.Sequential(
        nn.Linear(512*4,512),
        nn.LayerNorm(512),
        nn.ReLU(),
        nn.Linear(512,self.n_classes)
    )
      
  def forward(self,x_batch):
    word_embedding = self.bert(x_batch)[0]
    lstm_out,_=self.lstm(word_embedding)
    avg_hidden=torch.mean(lstm_out,dim=1)
    max_hidden,_=torch.max(lstm_out,dim=1)
    input_linear=torch.cat((avg_hidden,max_hidden),dim=1)
    out=self.linear(input_linear)
    return out

class LabelSmoothingCrossEntropyLoss(nn.Module):
    def __init__(self,smoothing_value=0.04,reduction='mean'):
        super(LabelSmoothingCrossEntropyLoss,self).__init__()
        self.smoothing_value=smoothing_value
        self.reduction=reduction
    
    def reduce_loss(self,loss,reduction='mean'):
        return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss 

    def forward(self,outputs,targets):
        n_classes=outputs.size(1)
        log_preds=F.log_softmax(outputs,dim=1)
        loss=self.reduce_loss(-log_preds.sum(dim=-1),self.reduction)
        nll=F.nll_loss(log_preds,targets,reduction=self.reduction)
        return (1-self.smoothing_value)*nll+ self.smoothing_value*(loss/n_classes)

In [20]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
n_epochs=100
num_classes=len(set(train.label))
# model_state=torch.load('/content/drive/MyDrive/Viettel3/Bert/model21.pth')
model=Model(num_classes).to(device)
optimizer=AdamW(model.parameters(),lr=1e-5)
scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=100,
    num_training_steps=len(train_loader)*(n_epochs-1)
)
scheduler_frozen=get_constant_schedule(optimizer)
criterion=LabelSmoothingCrossEntropyLoss().to(device)
frozen=True

In [ ]:
for child in phobert.children():
  for param in child.parameters():
    param.requires_grad = False

In [ ]:
def train_model(model,data_loader,scheduler_model):
  print("-----------------------Training--------------------------")
  model.train()
  loss_train=0
  final_preds=[]
  final_true=[]
  for idx,(x_batch,y_batch) in enumerate(data_loader):    
    x_batch=x_batch.to(device)
    y_batch=y_batch.to(device)
    outputs=model(x_batch)
    loss=criterion(outputs,y_batch)
    outputs=torch.softmax(outputs,dim=1)
    preds=torch.argmax(outputs,dim=1)
    final_preds.extend(preds.cpu().numpy().tolist())
    final_true.extend(y_batch.cpu().numpy().tolist())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler_model.step()
    loss_train+=loss.item()
    if idx%100==0:
      print(idx,end=' ')
  print()  
  return accuracy_score(final_true,final_preds),loss_train/len(data_loader)


def evaluate(model,data_loader):
  print("------------------------Evaluate---------------------------")
  model.eval()
  loss_val=0
  final_preds=[]
  final_true=[]
  for idx,(x_batch,y_batch) in enumerate(data_loader):
    x_batch=x_batch.to(device)
    y_batch=y_batch.to(device)
    outputs=model(x_batch)
    loss=criterion(outputs,y_batch)
    outputs=torch.softmax(outputs,dim=1)
    preds=torch.argmax(outputs,dim=1)
    loss_val+=loss.item()
    final_preds.extend(preds.cpu().numpy().tolist())
    final_true.extend(y_batch.cpu().numpy().tolist())
    if idx%100==0:
      print(idx,end=' ')

  print()
  return accuracy_score(final_true,final_preds),loss_val/len(data_loader)

In [ ]:
def testing(data_loader):
  model.eval()
  final_preds=[]
  final_true=[]
  with torch.no_grad():
    for idx,(x_batch,y_batch) in enumerate(data_loader):
      x_batch=x_batch.to(device)
      y_batch=y_batch.to(device)
      outputs=model(x_batch)
      outputs=torch.softmax(outputs,dim=1)
      preds=torch.argmax(outputs,dim=1)
      final_preds.extend(preds.cpu().numpy().tolist())
      final_true.extend(y_batch.cpu().numpy().tolist())
      if idx%100==0:
        print(idx,end=' ')
  print()
  print("Accuracy for test:",accuracy_score(final_true,final_preds))

def save_checkpoint(epoch,scheduler_model):
  path='/content/drive/MyDrive/Viettel3/Bert'
  if os.path.exists(path) is False:
    os.mkdir(path)
  
  with open('/content/drive/MyDrive/Viettel3/Bert/history{}.pickle'.format(epoch),'wb') as file:
    pickle.dump(history,file)
  
  print("Save history done")
  model_state={
      'model':model.state_dict(),
      'optimizer':optimizer.state_dict(),
      'scheduler':scheduler_model.state_dict(),
      'loss':criterion
  }
  torch.save(model_state,path+'/model{}.pth'.format(epoch))
  print("Save model done")

In [ ]:
from collections import defaultdict
import time 
history=defaultdict(list)
for epoch in range(n_epochs):

  if epoch > 0 and frozen:
    for child in model.bert.children():
      for param in child.parameters():
        param.requires_grad = False
    del scheduler_frozen
    torch.cuda.empty_cache()
    frozen=False 

  start_time=time.time()
  if frozen:
    train_acc,train_loss=train_model(model,train_loader,scheduler_frozen)
  else:
    train_acc,train_loss=train_model(model,train_loader,scheduler)
    
  val_acc,val_loss=evaluate(model,val_loader)
  print(f"Epoch {epoch}----Train acc:{train_acc}----Train loss:{train_loss}---Val acc:{val_acc}---Val loss:{val_loss}---Time:{time.time()-start_time}")
  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  if epoch!=0 and epoch%3==0:
    print("--------------Time for testing-----------------")
    testing(test_loader) 
    save_checkpoint(epoch,scheduler)

-----------------------Training--------------------------
0 100 200 300 400 500 600 700 800 
------------------------Evaluate---------------------------
0 100 200 
Epoch 0----Train acc:0.7784688286687398----Train loss:0.9281049807396161---Val acc:0.8636431006373203---Val loss:0.6226909455812373---Time:781.4636647701263
-----------------------Training--------------------------
0 100 200 300 400 500 600 700 800 
------------------------Evaluate---------------------------
0 100 200 
Epoch 1----Train acc:0.8759070042943877----Train loss:0.5760438872159586---Val acc:0.874907366236846---Val loss:0.5617461385320148---Time:784.7327320575714
-----------------------Training--------------------------
0 100 200 300 400 500 600 700 800 
------------------------Evaluate---------------------------
0 100 200 
Epoch 2----Train acc:0.8901969495039241----Train loss:0.5309797309206787---Val acc:0.8932859048465985---Val loss:0.5249709198825168---Time:785.463579416275
-----------------------Training--------

# Test Model

In [21]:
model_state=torch.load('/content/drive/MyDrive/Viettel3/Bert/model21.pth')
model=Model(n_classes=10).to(device)
model.load_state_dict(model_state['model'])
model.eval()

Model(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [22]:
def evaluate(data_loader):
  final_preds=[]
  final_true=[]
  for idx,(x_batch,y_batch) in enumerate(data_loader):
    x_batch=x_batch.to(device)
    y_batch=y_batch.to(device)
    outputs=model(x_batch)
    outputs=torch.softmax(outputs,dim=1)
    preds=torch.argmax(outputs,dim=1)
    final_preds.extend(preds.cpu().numpy().tolist())
    final_true.extend(y_batch.cpu().numpy().tolist())
    if idx%100==0:
      print(idx,end=' ')
  return final_true,final_preds

In [24]:
final_true_val,final_preds_val=evaluate(val_loader)

0 100 200 300 400 

In [25]:
final_true_test,final_true_preds=evaluate(test_loader)

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 

In [27]:
def show_result(y_true,y_preds):
  print(accuracy_score(y_true,y_preds))
  print(precision_score(y_true,y_preds,average='weighted'))
  print(recall_score(y_true,y_preds,average='weighted'))
  print(f1_score(y_true,y_preds,average='weighted'))

In [28]:
print(classification_report(final_true_val,final_preds_val,target_names=list(labels.values())))

                  precision    recall  f1-score   support

         Van hoa       0.95      0.93      0.94       616
Chinh tri Xa hoi       0.87      0.81      0.84      1043
        Khoa hoc       0.91      0.79      0.85       364
       Phap luat       0.92      0.93      0.93       773
         Vi tinh       0.96      0.95      0.95       496
        Suc khoe       0.89      0.97      0.93       676
      Kinh doanh       0.82      0.92      0.87       510
        The thao       0.99      0.99      0.99      1059
        The gioi       0.95      0.90      0.92       579
        Doi song       0.85      0.91      0.88       631

        accuracy                           0.91      6747
       macro avg       0.91      0.91      0.91      6747
    weighted avg       0.91      0.91      0.91      6747



In [29]:
print(classification_report(final_true_test,final_true_preds,target_names=list(labels.values())))

                  precision    recall  f1-score   support

         Van hoa       0.93      0.95      0.94      6250
Chinh tri Xa hoi       0.89      0.85      0.87      7567
        Khoa hoc       0.85      0.75      0.80      2096
       Phap luat       0.89      0.92      0.91      3788
         Vi tinh       0.94      0.95      0.94      4560
        Suc khoe       0.91      0.95      0.93      5417
      Kinh doanh       0.87      0.92      0.89      5276
        The thao       0.98      0.99      0.98      6667
        The gioi       0.96      0.92      0.94      6716
        Doi song       0.72      0.70      0.71      2036

        accuracy                           0.91     50373
       macro avg       0.89      0.89      0.89     50373
    weighted avg       0.91      0.91      0.91     50373



In [30]:
show_result(final_true_test,final_true_preds)

0.9122148770174499
0.9119381121434791
0.9122148770174499
0.9116690540717909


In [ ]:
def plot(final_true,final_preds,file_name):
  temp=confusion_matrix(final_true,final_preds)
  df=pd.DataFrame(temp,index=labels.values(),columns=labels.values()).astype(int)
  plt.figure(figsize=(12,8))
  heatmap=sns.heatmap(df,annot=True,fmt="d")
  heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(),rotation=0,ha='right')
  heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(),rotation=45,ha='right')
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  result=accuracy_score(final_true,final_preds)
  plt.title("Accuracy for test:{:0.5f}".format(result))
  plt.savefig('/content/drive/MyDrive/Viettel3/Bert/{}'.format(file_name))

In [ ]:
plot(final_true_val,final_preds_val,'bert_val.png')

In [ ]:
plot(final_true_test,final_true_preds,'bert_test.png')